In [1]:
import datetime
import pandas as pd
import requests
import io
import json

url = 'http://gis.sutran.gob.pe/alerta_sutran/script_cgm/carga_xlsx.php'
headers = {'Accept': '/',
'Accept-Encoding': 'gzip, deflate',
'Accept-Language': 'en,es;q=0.9,en-GB;q=0.8',
'Connection': 'keep-alive',
'Content-Length': '9',
'Content-type': 'application/x-www-form-urlencoded; charset=UTF-8-SIG',
'Host': 'gis.sutran.gob.pe',
'Origin': 'http://gis.sutran.gob.pe',
'Referer': 'http://gis.sutran.gob.pe/alerta_sutran/',
'User-Agent': 'Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Mobile Safari/537.36'}
body = {'tipo':'MAPA'}
req = requests.post(url, headers=headers, data=body)
now = datetime.datetime.now()

my_json = json.load(io.BytesIO(req.content))

restringido = {}
restringido['data'] = my_json['restringido']

interrumpido = {}
interrumpido['data'] = my_json['interrumpido']

with open('restringido.json', 'w') as f:
    json.dump(restringido, f)

with open('interrumpido.json', 'w') as f:
    json.dump(interrumpido, f)

df_r = pd.read_json('restringido.json', encoding = 'utf-8-sig', orient='split')
df_restringido = pd.json_normalize(df_r['properties'])

df_i = pd.read_json('interrumpido.json', encoding = 'utf-8-sig', orient='split')
df_interrumpido = pd.json_normalize(df_i['properties'])

frames = [df_restringido, df_interrumpido]
df_total = pd.concat(frames, ignore_index=True)
df_total = df_total[df_total['motivo'] == 'HUMANO']
df_total = df_total.replace('TRANSITO', 'TRÁNSITO', regex=True)
df_total = df_total.replace('"', '', regex=True)
df_total['style'] = df_total['estado'].str.split(' ').str[1].str.lower()
df_total.to_csv(f'{now}.csv', index=False)
df_total

,item,visualizacion_,tipo_alerta_,estado,fecha_evento,fecha_actualizacion,ubigeo,afectacion,evento,latitud,...,_sort_,_fecha_actualizacion_,cant_vehiculos_detenidos_pasajeros,cant_vehiculos_detenidos_mercancias,fuente,motivo,codigo_via,nombre_carretera,alerta_activa_24_horas,style
2,216,SI,VIGENTE,TRÁNSITO INTERRUMPIDO,31/01/2023,31/01/2023 17:30 HORAS,JUNIN/CHANCHAMAYO/PICHANAQUI,KM 69+200,PARO INDEFINIDO SOLICITANDO NUEVAS ELECCIONES ...,-10.92402,...,-10.92402+-74.877533,2023 17:30 HORAS+01+31+17:30,0,0,COE-MTC,HUMANO,PE-5S,LONGITUDINAL DE LA SELVA SUR,SI,interrumpido
3,259,SI,VIGENTE,TRÁNSITO INTERRUMPIDO,06/01/2023,31/01/2023 17:30 HORAS,MADRE DE DIOS/TAMBOPATA/LAS PIEDRAS,KM 511+800,(PUENTE MAVILA) PARO INDEFINIDO SOLICITANDO NU...,-11.931332,...,-11.931332+-69.11748,2023 17:30 HORAS+01+31+17:30,0,0,IIRSA SUR,HUMANO,PE-30C,CARRETERA URCOS - MAZUCO - IÑAPARI,SI,interrumpido
4,257,SI,VIGENTE,TRÁNSITO INTERRUMPIDO,30/01/2023,31/01/2023 17:30 HORAS,MADRE DE DIOS/TAMBOPATA/LAS PIEDRAS,KM 471,(SECTOR PLANCHON) PARO INDEFINIDO SOLICITANDO...,-12.2828444,...,-12.2828444+-69.1554046,2023 17:30 HORAS+01+31+17:30,0,0,IIRSA SUR,HUMANO,PE-30C,CARRETERA URCOS - MAZUCO - IÑAPARI,SI,interrumpido
5,252,SI,VIGENTE,TRÁNSITO INTERRUMPIDO,20/01/2023,31/01/2023 17:30 HORAS,MADRE DE DIOS/TAMBOPATA/LABERINTO,KM 382+200,(SECTOR LABERINTO) PARO INDEFINIDO SOLICITANDO...,-12.7637802,...,-12.7637802+-69.5700566,2023 17:30 HORAS+01+31+17:30,2,4,IIRSA SUR,HUMANO,PE-30C,CARRETERA URCOS - MAZUCO - IÑAPARI,SI,interrumpido
6,248,SI,VIGENTE,TRÁNSITO INTERRUMPIDO,04/01/2023,31/01/2023 17:30 HORAS,MADRE DE DIOS/TAMBOPATA/INAMBARI,KM 288-289,(SECTOR SANTA ROSA) PARO INDEFINIDO SOLICITAND...,-12.932413,...,-12.932413+-70.303984,2023 17:30 HORAS+01+31+17:30,0,0,IIRSA SUR,HUMANO,PE-30C,CARRETERA URCOS - MAZUCO - IÑAPARI,SI,interrumpido
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77,297,SI,VIGENTE,TRÁNSITO INTERRUMPIDO,05/01/2023,31/01/2023 17:30 HORAS,PUNO/EL COLLAO/SANTA ROSA,KM 175 AL 213,PARO INDEFINIDO SOLICITANDO NUEVAS ELECCIONES ...,-16.7501666,...,-16.7501666+-69.7209405,2023 17:30 HORAS+01+31+17:30,0,0,CEOPOL,HUMANO,PE-36A,CARRETERA MOQUEGUA - TORATA - MAZOCRUZ - DESAG...,SI,interrumpido
78,292,SI,VIGENTE,TRÁNSITO INTERRUMPIDO,07/01/2023,31/01/2023 17:30 HORAS,PUNO/EL COLLAO/CAPAZO,KM 171 AL 226,PARO INDEFINIDO SOLICITANDO NUEVAS ELECCIONES ...,-17.19684952581,...,-17.19684952581+-69.759059671272,2023 17:30 HORAS+01+31+17:30,0,0,COE-MTC,HUMANO,PE-36A,CARRETERA MOQUEGUA - TORATA - MAZOCRUZ - DESAG...,SI,interrumpido
79,108,SI,VIGENTE,TRÁNSITO INTERRUMPIDO,31/01/2023,31/01/2023 17:30 HORAS,CAJAMARCA/CHOTA/LLAMA,KM 84,(PUENTE CUMBIL) PARO SOLICITANDO NUEVAS ELECC...,-6.60115016,...,-6.60115016+-79.24774852,2023 17:30 HORAS+01+31+17:30,0,0,CEOPOL CAJAMARCA,HUMANO,PE-06A,CARRETERA CHICLAYO - HUAMBOS - COCHABAMBA,SI,interrumpido
80,101,SI,VIGENTE,TRÁNSITO INTERRUMPIDO,31/01/2023,31/01/2023 17:30 HORAS,CAJAMARCA/CONTUMAZA/CHILETE,KM 89,PARO INDEFINIDO SOLICITANDO NUEVAS ELECCIONES ...,-7.225185923,...,-7.225185923+-78.85536843,2023 17:30 HORAS+01+31+17:30,0,0,CONCESIONARIA CONVIAL SIERRA NORTE,HUMANO,PE-08,CARRETERA PACASMAYO - CHILETE - CAJAMARCA,SI,interrumpido
